In [1]:
# TO DO
#
# MVP: create a version that allows us to pass and no more

Copied from assignment 2. NEEDS WORK!!!

In [2]:
# Organize data for pytorch DataLoader
import os
import shutil
import random
import torch

#DATA_DIR = 'D:/Kurssit/Data Science/Deep learning/dl2021-image-corpus-proj/'
DATA_DIR = '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/'
ANNOTATIONS_DIR = DATA_DIR + 'annotations/'
IMAGES_DIR = DATA_DIR + 'images/'

# New fodlers for train, test, and dev sets
TRAIN_DIR = DATA_DIR + 'train/'
DEV_DIR = DATA_DIR + 'dev/'
TEST_DIR = DATA_DIR + 'test/'

annotations = ["baby",
               "bird",
               "car",
               "clouds",
               "dog",
               "female",
               "flower",
               "male",
               "night",
               "people",
               "portrait",
               "river",
               "sea",
               "tree"]

/Users/hartih/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create labels for images
image_file_names = os.listdir(IMAGES_DIR)
dict_labels = {}
for image_file_name in image_file_names:  # Initiate label tensors
    if os.path.isfile(IMAGES_DIR + image_file_name):
        dict_labels[image_file_name] = torch.zeros(14)
for i in range(len(annotations)):  # Fill label tensors with 1's if found in one of the annotations text files
    with open(ANNOTATIONS_DIR + annotations[i] + ".txt") as f:
        for row in f:
            row = "im" + row.strip() + ".jpg"
            dict_labels[row][i] = 1

In [4]:
dict_labels

{'im1976.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im12710.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im14361.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im17668.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im10107.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im11219.jpg': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im19645.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im16576.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im19123.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im16210.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'im10661.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im3807.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im8952.jpg': ten

In [5]:
# Split the data to train, test, and dev
os.makedirs(TRAIN_DIR)
os.makedirs(TEST_DIR)
os.makedirs(DEV_DIR)
for image_file_name in image_file_names:
    if os.path.isfile(IMAGES_DIR + image_file_name):
        division = random.randint(1, 3)
        if division == 1:
            shutil.copyfile(IMAGES_DIR + image_file_name, TRAIN_DIR + image_file_name)
        if division == 2:
            shutil.copyfile(IMAGES_DIR + image_file_name, DEV_DIR + image_file_name)
        if division == 3:
            shutil.copyfile(IMAGES_DIR + image_file_name, TEST_DIR + image_file_name)

FileExistsError: [Errno 17] File exists: '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/train/'

In [6]:
print( len(os.listdir(TEST_DIR)) + len(os.listdir(TRAIN_DIR)) + len( os.listdir(DEV_DIR)) == len(os.listdir(IMAGES_DIR)) )
print(len(os.listdir(TEST_DIR)))
print(len(os.listdir(TRAIN_DIR)))
print(len(os.listdir(DEV_DIR)))
print(len(os.listdir(IMAGES_DIR)))


True
6786
6697
6517
20000


In [7]:
import torch.utils.data
from PIL import Image

# Enable creating train, test, and dev test datasets for PyTorch
class myDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir=IMAGES_DIR, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = [root_dir + img for img in os.listdir(root_dir)]                
      
    def __len__(self):
        return len(self.images)       

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = img = Image.open(img_path)     
       
        if self.transform:
            img = self.transform(img)     

        return img, dict_labels[img_path.split("/")[-1]]

In [8]:
from torchvision import transforms
import numpy as np

#Calculate mean and std with whole dataset
imageNet = True  # If imageNet is True, then we use the values from imagenet, else we calculate them from the dataset

if imageNet:
  mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]  # Imagenet mean and std
else:
  def mean_std(loader):
    images, labels = next(iter(loader))
    # shape of images = [b,c,w,h]
    mean, std = images.mean([0,2,3]), images.std([0,2,3])
    return mean, std

  mean_transform = transforms.Compose([
                                          transforms.Grayscale(num_output_channels=3),
                                          transforms.Resize(256),                    
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor()])

  #Splitting these was the easiest due to memory allocation problem, you could only used IMAGES_DIR if this is not a problem, note the equal divide
  train_mean_set = myDataset(TRAIN_DIR, transform=mean_transform)
  train_data_loader = torch.utils.data.DataLoader(dataset=train_mean_set, batch_size=len(train_mean_set), shuffle=False, num_workers=0)
  dev_mean_set = myDataset(DEV_DIR, transform=mean_transform)
  dev_data_loader = torch.utils.data.DataLoader(dataset=dev_mean_set, batch_size=len(dev_mean_set), shuffle=False, num_workers=0)
  test_mean_set = myDataset(TEST_DIR, transform=mean_transform)
  test_data_loader = torch.utils.data.DataLoader(dataset=test_mean_set, batch_size=len(test_mean_set), shuffle=False, num_workers=0)

  mean_train, std_train = mean_std(train_data_loader)
  mean_dev, std_dev = mean_std(dev_data_loader)
  mean_test, std_test = mean_std(test_data_loader)
  mean = list((np.array(mean_train)+np.array(mean_dev)+np.array(mean_test))/3)
  std = list((np.array(std_train)+np.array(std_dev)+np.array(std_test))/3)
print("Mean:", mean)
print("std:", std)

#Imagenet mean and std
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
                                        #transforms.RandomHorizontalFlip(),
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.RandomCrop(128),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(2),
                                        transforms.ToTensor()])
test_transform = transforms.Compose([
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor()])

transform = transforms.Compose([            #[1]
 transforms.Grayscale(num_output_channels=3),
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=mean,                #[6]
 std=std                 #[7]
 )]
)

# Create datasets for CNN
train_set = myDataset(TRAIN_DIR, transform=transform)
test_set = myDataset(TEST_DIR, transform=transform)
dev_set = myDataset(DEV_DIR, transform=transform)

Mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]


In [9]:
import torch 

# Calculate the amount of correctly predicted as well as total predictions done
def calc_correct(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)  # Since our neural network does not apply sigmoid
    correct_dict = {'tot': [0,0]}  # First number in value is correct ones, the second one is total amount
    correct_dict['tot_strict'] = [0,0] # All correct
    for i in range(len(pred)): # [100,14]
        all_correct = 0
        total = 0
        # estim_pred_array = []
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            # estim_pred_array.append(estim_pred)
            if annotations[j] not in correct_dict.keys():
                correct_dict[annotations[j]] = [0,0]
            correct_dict['tot'][1] += 1
            correct_dict[annotations[j]][1] += 1
            correct_dict['tot'][0] += int(estim_pred == target[i][j])
            correct_dict[annotations[j]][0] += int(estim_pred == target[i][j])
            all_correct += int(estim_pred == target[i][j])
            total += 1
        correct_dict['tot_strict'][1] += 1
        correct_dict['tot_strict'][0] += int(total==all_correct)
    # To-do how many pictures were entirely correct (accuracy)
    return correct_dict


In [10]:
def class_evaluation(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    negative = 0
    positive = 0
    
    for i in range(len(pred)): # [100,14]
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                negative += 1
                if estim_pred == 1:
                    false_positive += 1
                if estim_pred == 0:
                    true_negative += 1
            
            # Positive target values
            if target[i][j] == 1:
                positive += 1
                if estim_pred == 1:
                    true_positive += 1
                if estim_pred == 0:
                    false_negative += 1
    
    result = {"true_positive": true_positive,
            "false_positive": false_positive,
            "true_negative": true_negative,
            "false_negative": false_negative,
            "negative": negative,
            "positive": positive}
                    
    return result

In [11]:
def class_evaluation_by_annotation(pred: torch.Tensor, target: torch.Tensor):
    
    pred = torch.sigmoid(pred)
    
    # Initiate vslues for every annotation
    eval_dict = {}
    for a in range(len(annotations)):
        eval_dict[annotations[a]] = {"true_positive": 0,
                                        "false_positive": 0,
                                        "true_negative": 0,
                                        "false_negative": 0,
                                        "negative": 0,
                                        "positive": 0}
    
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                eval_dict[annotations[j]]["negative"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["false_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["true_negative"] += 1
            
            # Positive target values
            if target[i][j] == 1:
                eval_dict[annotations[j]]["positive"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["true_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["false_negative"] += 1
                    
    return eval_dict

In [ ]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets, models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


#--- hyperparameters ---
N_EPOCHS = 25
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.001
WEIGHT_DECAY = 0.00
MOMENTUM = 0.1


#--- fixed constants ---
NUM_CLASSES = 14
NUM_CHANNELS = 3



# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.


# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, shuffle=False)

alexnet = models.alexnet(pretrained=True)
resnet = models.resnet101(pretrained=True)

alexnet.train()
resnet.train()

#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        
        self.alex = alexnet
        for p in self.alex.parameters():
            p.requires_grad=False
        
        self.resnet = resnet
        for p in self.resnet.parameters():
            p.requires_grad=False
        
        self.lin = nn.Sequential(
            nn.Linear(2000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 250),
            nn.ReLU(),
            nn.Linear(250, num_classes)
        )

    def forward(self, x):
        
        out_alex = self.alex(x)
        out_resnet = self.resnet(x)
        
        combined_pretrained_output = torch.cat((out_alex, out_resnet), dim=1)
        
        out = self.lin(combined_pretrained_output)
        
        return out



#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
loss_function = nn.BCEWithLogitsLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
stop_early = False

#--- training ---
for epoch in range(N_EPOCHS):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    train_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}
    total = 0

    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        train_correct['tot_strict'][0] += new_correct['tot_strict'][0]
        train_correct['tot_strict'][1] += new_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]

        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1],
               100. * train_correct['tot_strict'][0] / train_correct['tot_strict'][1], train_correct['tot_strict'][0], train_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

    cur_dev_loss = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    dev_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            dev_correct['tot_strict'][0] += new_dev_correct['tot_strict'][0]
            dev_correct['tot_strict'][1] += new_dev_correct['tot_strict'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        current_accuracy['tot_strict'] = 100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1]
        dev_accuracies.append(current_accuracy)

        if current_loss <= dev_loss:
            dev_loss = current_loss
        # else:
        #     stop_early = True

        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1],
            100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1], dev_correct['tot_strict'][0], dev_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")


------------------------
Training: Epoch 1 - Batch 1/67: Loss: 0.7020 | Train Acc: 44.571% (624/1400) | Strict Acc: 0.000% (0/100)
True positive rate: 65.116% (56/86)
False negative rate: 34.884% (30/86)
True negative rate: 43.227% (568/1314)
False positive rate: 56.773% (746/1314)
------------------------
------------------------
Training: Epoch 1 - Batch 2/67: Loss: 0.5554 | Train Acc: 66.143% (1852/2800) | Strict Acc: 9.500% (19/200)
True positive rate: 30.729% (59/192)
False negative rate: 69.271% (133/192)
True negative rate: 68.750% (1793/2608)
False positive rate: 31.250% (815/2608)
------------------------
------------------------
Training: Epoch 1 - Batch 3/67: Loss: 0.4533 | Train Acc: 75.143% (3156/4200) | Strict Acc: 21.333% (64/300)
True positive rate: 26.246% (79/301)
False negative rate: 73.754% (222/301)
True negative rate: 78.918% (3077/3899)
False positive rate: 21.082% (822/3899)
------------------------
------------------------
Training: Epoch 1 - Batch 4/67: Loss: 

------------------------
Training: Epoch 1 - Batch 27/67: Loss: 0.2007 | Train Acc: 92.272% (34879/37800) | Strict Acc: 47.481% (1282/2700)
True positive rate: 41.566% (1141/2745)
False negative rate: 58.434% (1604/2745)
True negative rate: 96.243% (33738/35055)
False positive rate: 3.757% (1317/35055)
------------------------
------------------------
Training: Epoch 1 - Batch 28/67: Loss: 0.1982 | Train Acc: 92.360% (36205/39200) | Strict Acc: 47.679% (1335/2800)
True positive rate: 41.891% (1196/2855)
False negative rate: 58.109% (1659/2855)
True negative rate: 96.324% (35009/36345)
False positive rate: 3.676% (1336/36345)
------------------------
------------------------
Training: Epoch 1 - Batch 29/67: Loss: 0.1958 | Train Acc: 92.470% (37543/40600) | Strict Acc: 47.862% (1388/2900)
True positive rate: 42.284% (1244/2942)
False negative rate: 57.716% (1698/2942)
True negative rate: 96.391% (36299/37658)
False positive rate: 3.609% (1359/37658)
------------------------
-------------

------------------------
Training: Epoch 1 - Batch 52/67: Loss: 0.1629 | Train Acc: 93.695% (68210/72800) | Strict Acc: 51.231% (2664/5200)
True positive rate: 47.167% (2489/5277)
False negative rate: 52.833% (2788/5277)
True negative rate: 97.331% (65721/67523)
False positive rate: 2.669% (1802/67523)
------------------------
------------------------
Training: Epoch 1 - Batch 53/67: Loss: 0.1621 | Train Acc: 93.725% (69544/74200) | Strict Acc: 51.283% (2718/5300)
True positive rate: 47.441% (2549/5373)
False negative rate: 52.559% (2824/5373)
True negative rate: 97.338% (66995/68827)
False positive rate: 2.662% (1832/68827)
------------------------
------------------------
Training: Epoch 1 - Batch 54/67: Loss: 0.1617 | Train Acc: 93.733% (70862/75600) | Strict Acc: 51.093% (2759/5400)
True positive rate: 47.358% (2590/5469)
False negative rate: 52.642% (2879/5469)
True negative rate: 97.349% (68272/70131)
False positive rate: 2.651% (1859/70131)
------------------------
-------------

------------------------
Training: Epoch 2 - Batch 10/67: Loss: 0.1016 | Train Acc: 96.307% (13483/14000) | Strict Acc: 64.400% (644/1000)
True positive rate: 61.990% (592/955)
False negative rate: 38.010% (363/955)
True negative rate: 98.819% (12891/13045)
False positive rate: 1.181% (154/13045)
------------------------
------------------------
Training: Epoch 2 - Batch 11/67: Loss: 0.1014 | Train Acc: 96.312% (14832/15400) | Strict Acc: 64.364% (708/1100)
True positive rate: 62.677% (665/1061)
False negative rate: 37.323% (396/1061)
True negative rate: 98.800% (14167/14339)
False positive rate: 1.200% (172/14339)
------------------------
------------------------
Training: Epoch 2 - Batch 12/67: Loss: 0.1033 | Train Acc: 96.268% (16173/16800) | Strict Acc: 63.833% (766/1200)
True positive rate: 62.650% (733/1170)
False negative rate: 37.350% (437/1170)
True negative rate: 98.784% (15440/15630)
False positive rate: 1.216% (190/15630)
------------------------
------------------------
Tr

------------------------
Training: Epoch 2 - Batch 36/67: Loss: 0.1056 | Train Acc: 95.990% (48379/50400) | Strict Acc: 61.972% (2231/3600)
True positive rate: 61.858% (2251/3639)
False negative rate: 38.142% (1388/3639)
True negative rate: 98.646% (46128/46761)
False positive rate: 1.354% (633/46761)
------------------------
------------------------
Training: Epoch 2 - Batch 37/67: Loss: 0.1057 | Train Acc: 95.981% (49718/51800) | Strict Acc: 62.000% (2294/3700)
True positive rate: 61.851% (2312/3738)
False negative rate: 38.149% (1426/3738)
True negative rate: 98.635% (47406/48062)
False positive rate: 1.365% (656/48062)
------------------------
------------------------
Training: Epoch 2 - Batch 38/67: Loss: 0.1054 | Train Acc: 95.992% (51068/53200) | Strict Acc: 62.000% (2356/3800)
True positive rate: 62.146% (2392/3849)
False negative rate: 37.854% (1457/3849)
True negative rate: 98.632% (48676/49351)
False positive rate: 1.368% (675/49351)
------------------------
----------------

...

------------------------
Training: Epoch 18 - Batch 28/67: Loss: 0.0222 | Train Acc: 99.166% (38873/39200) | Strict Acc: 90.143% (2524/2800)
True positive rate: 93.933% (2787/2967)
False negative rate: 6.067% (180/2967)
True negative rate: 99.594% (36086/36233)
False positive rate: 0.406% (147/36233)
------------------------
------------------------
Training: Epoch 18 - Batch 29/67: Loss: 0.0223 | Train Acc: 99.163% (40260/40600) | Strict Acc: 90.103% (2613/2900)
True positive rate: 93.991% (2878/3062)
False negative rate: 6.009% (184/3062)
True negative rate: 99.584% (37382/37538)
False positive rate: 0.416% (156/37538)
------------------------
------------------------
Training: Epoch 18 - Batch 30/67: Loss: 0.0223 | Train Acc: 99.160% (41647/42000) | Strict Acc: 90.033% (2701/3000)
True positive rate: 94.015% (2969/3158)
False negative rate: 5.985% (189/3158)
True negative rate: 99.578% (38678/38842)
False positive rate: 0.422% (164/38842)
------------------------
-------------------

------------------------
Training: Epoch 18 - Batch 54/67: Loss: 0.0235 | Train Acc: 99.101% (74920/75600) | Strict Acc: 89.593% (4838/5400)
True positive rate: 93.365% (5094/5456)
False negative rate: 6.635% (362/5456)
True negative rate: 99.547% (69826/70144)
False positive rate: 0.453% (318/70144)
------------------------
------------------------
Training: Epoch 18 - Batch 55/67: Loss: 0.0236 | Train Acc: 99.096% (76304/77000) | Strict Acc: 89.564% (4926/5500)
True positive rate: 93.349% (5193/5563)
False negative rate: 6.651% (370/5563)
True negative rate: 99.544% (71111/71437)
False positive rate: 0.456% (326/71437)
------------------------
------------------------
Training: Epoch 18 - Batch 56/67: Loss: 0.0238 | Train Acc: 99.084% (77682/78400) | Strict Acc: 89.429% (5008/5600)
True positive rate: 93.215% (5289/5674)
False negative rate: 6.785% (385/5674)
True negative rate: 99.542% (72393/72726)
False positive rate: 0.458% (333/72726)
------------------------
-------------------

------------------------
Training: Epoch 19 - Batch 12/67: Loss: 0.0197 | Train Acc: 99.292% (16681/16800) | Strict Acc: 91.750% (1101/1200)
True positive rate: 95.532% (1176/1231)
False negative rate: 4.468% (55/1231)
True negative rate: 99.589% (15505/15569)
False positive rate: 0.411% (64/15569)
------------------------
------------------------
Training: Epoch 19 - Batch 13/67: Loss: 0.0199 | Train Acc: 99.280% (18069/18200) | Strict Acc: 91.692% (1192/1300)
True positive rate: 95.347% (1250/1311)
False negative rate: 4.653% (61/1311)
True negative rate: 99.586% (16819/16889)
False positive rate: 0.414% (70/16889)
------------------------
------------------------
Training: Epoch 19 - Batch 14/67: Loss: 0.0198 | Train Acc: 99.276% (19458/19600) | Strict Acc: 91.571% (1282/1400)
True positive rate: 95.061% (1328/1397)
False negative rate: 4.939% (69/1397)
True negative rate: 99.599% (18130/18203)
False positive rate: 0.401% (73/18203)
------------------------
------------------------


------------------------
Training: Epoch 19 - Batch 38/67: Loss: 0.0213 | Train Acc: 99.205% (52777/53200) | Strict Acc: 90.605% (3443/3800)
True positive rate: 94.094% (3537/3759)
False negative rate: 5.906% (222/3759)
True negative rate: 99.593% (49240/49441)
False positive rate: 0.407% (201/49441)
------------------------
------------------------
Training: Epoch 19 - Batch 39/67: Loss: 0.0211 | Train Acc: 99.207% (54167/54600) | Strict Acc: 90.641% (3535/3900)
True positive rate: 94.139% (3646/3873)
False negative rate: 5.861% (227/3873)
True negative rate: 99.594% (50521/50727)
False positive rate: 0.406% (206/50727)
------------------------
------------------------
Training: Epoch 19 - Batch 40/67: Loss: 0.0211 | Train Acc: 99.204% (55554/56000) | Strict Acc: 90.575% (3623/4000)
True positive rate: 94.095% (3729/3963)
False negative rate: 5.905% (234/3963)
True negative rate: 99.593% (51825/52037)
False positive rate: 0.407% (212/52037)
------------------------
-------------------

------------------------
Training: Epoch 19 - Batch 64/67: Loss: 0.0221 | Train Acc: 99.167% (88854/89600) | Strict Acc: 90.141% (5769/6400)
True positive rate: 93.882% (6077/6473)
False negative rate: 6.118% (396/6473)
True negative rate: 99.579% (82777/83127)
False positive rate: 0.421% (350/83127)
------------------------
------------------------
Training: Epoch 19 - Batch 65/67: Loss: 0.0222 | Train Acc: 99.167% (90242/91000) | Strict Acc: 90.138% (5859/6500)
True positive rate: 93.883% (6155/6556)
False negative rate: 6.117% (401/6556)
True negative rate: 99.577% (84087/84444)
False positive rate: 0.423% (357/84444)
------------------------
------------------------
Training: Epoch 19 - Batch 66/67: Loss: 0.0223 | Train Acc: 99.165% (91628/92400) | Strict Acc: 90.106% (5947/6600)
True positive rate: 93.874% (6268/6677)
False negative rate: 6.126% (409/6677)
True negative rate: 99.577% (85360/85723)
False positive rate: 0.423% (363/85723)
------------------------
-------------------

In [ ]:
#--- test ---

alexnet.eval()
resnet.eval()

test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
test_correct['tot_strict'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        test_correct['tot_strict'][0] += new_test_correct['tot_strict'][0]
        test_correct['tot_strict'][1] += new_test_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1],
               100. * test_correct['tot_strict'][0] / test_correct['tot_strict'][1], test_correct['tot_strict'][0], test_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

In [ ]:
for i in evaluation_by_annotation:
    print("------------------------")
    print(i + ":")
    print("")
    print("positive: " + str(evaluation_by_annotation[i]["positive"]))
    print("true_positive: " + str(evaluation_by_annotation[i]["true_positive"]))
    print("false_negative: " + str(evaluation_by_annotation[i]["false_negative"]))
    print("")
    print("negative: " + str(evaluation_by_annotation[i]["negative"]))
    print("true_negative: " + str(evaluation_by_annotation[i]["true_negative"]))
    print("false_positive: " + str(evaluation_by_annotation[i]["false_positive"]))
        
